listes de q pure

#### Aiida Lab data importation

In [3]:
#entre liste des pression et liste des loading_absolute_average, les transfo en csv file puis les importe sur ce doc
def conv_to_csv(pressure,loading_absolute_average,mof_gas):
    df = pd.DataFrame({
        'Pressure (bar)': pressure,
        'Loading (mmol/g)': loading_absolute_average
    })

    # Save data into a csv file
    df.to_csv('isotherm_data_{mof_gas}.csv', index=False)
#    return

    #import data from the csv file
#def import_data(mof_gas):
    df_mof = pd.read_csv("isotherm_data_{mof_gas}.csv")
    return df_mof

#### Loading q values in pure conditions

In [1]:
def get_pure_q(df_mof):
    pure_isotherm = pyiast.InterpolatorIsotherm(df_mof,
                                        loading_key="Loading (mmol/g)",
                                        pressure_key="Pressure (bar)")
    pure_isotherm = pyiast.InterpolatorIsotherm(df_mof,
                                        loading_key="Loading (mmol/g)",
                                        pressure_key="Pressure (bar)",
                                        fill_value=df_mof['Loading (mmol/g)'].max())
    return pure_isotherm

#### Loading q values in binary mixture

In [4]:
def get_mixt_q(N2_isotherm, CO2_isotherm, pressure):
    p_part = pressure*np.array([0.85,0.15])
    q_mix = pyiast.iast(p_part, [N2_isotherm, CO2_isotherm], verboseflag=False)
    return q_mix

#### selectivity calculation

In [7]:
def get_selectivity(mof,pressures,loading_absolute_average, S_pressure):
    name_N2 = f"{mof}_N2"
    name_CO2 = f"{mof}_CO2"
    
    df_N2 = conv_to_csv(pressures, loading_absolute_average, name_N2)
    N2_isotherm = get_pure_q(df_N2)

    df_CO2 = conv_to_csv(pressures, loading_absolute_average, name_CO2)
    CO2_isotherm = get_pure_q(df_CO2)
    
    q_mix = get_mixt_q(N2_isotherm, CO2_isotherm, S_pressure)
    Selectivity_mof = (0.85/0.15)*(q_mix[1]/q_mix[2])
    return Selectivity_mof

#### values computation and display